In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt# data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LOADING THE DATA SET**

In [ ]:
df=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")


***CLEANING THE DATA***
***METHOD 1-HANDLING MISSING VALUES BY DELETING THE ROWS AND COLUMNS***

In [ ]:
df.shape

In [ ]:
df.head(6)
#Here,you can see that not all columns are visible since there are 81 of them.Therefore we use the function set_option to set the data to display all the columns and rows as is done below

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
df.head(6)

In [ ]:
df.info()
#From this frunction, we can see that there are a lot of missing values in the data set.For example,in the column Alley, only 91 non-null objects are present out of 1460, and many more.

In [ ]:
#Since,the df.info() function's result becomes a bit difficult to examine for a huge data set we use the function isnull() and also sum()
df.shape
#df.isnull()returns False wherever there is a value and True wherever there is no value
#To make it more convinient, we use df.isnull().sum() as it returns the sum of all null values in each column respectively

In [ ]:
df.isnull().sum()


In [ ]:
#Because our brain understands what we visualize better, we will plot this sum of non nul values into a heatmap
plt.figure(figsize=(25,25))
sns.heatmap(df.isnull())


In [ ]:
#We will find the percentage of data missing from each column and then drop the column which has most data missing.For example,from the heatmap we can clearly see that the Column Alley has most of its data missing.
null_var= df.isnull().sum()/df.shape[0] *100
null_var

In [ ]:
drop_columns  = null_var[null_var >17].keys()
drop_columns

In [ ]:
dfnew= df.drop(columns=drop_columns)

In [ ]:
sns.heatmap(dfnew.isnull())
#we can now see that the columns which had alot off null values have no wbeen removed but there are still some missing values left in the data set


In [ ]:
#Now we will delete all the rows which have missing values as Nan
df3=dfnew.dropna()
df3.shape

In [ ]:
sns.heatmap(df3.isnull())
#now we can see that there are no missing values in the dataframe


In [ ]:
df3.isnull().sum().sum()
#no missing values left in numeric term also

In [ ]:
#now to see if we have handled the data correctly,we will see the distribution of the data
#Distribution of the data can be with numeric data only,hence we first select all the data that has numeric values
num_columns=df3.select_dtypes(include=['int64','float64']).columns
num_columns

In [ ]:
sns.distplot(df['MSSubClass'])

In [ ]:
sns.distplot(df3['MSSubClass'])

In [ ]:
sns.distplot(df['MSSubClass'])
sns.distplot(df3['MSSubClass'])
#We have overlapped the two plots to see how much difference is there.
#We can see that there is not much difference in the two plots.It means we have kind of handled the data correctly

In [ ]:
#Now, we have to check for all numeric columns, so we will make a list of all column names and then do the needful
variable=['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice']
#Note that column name Id will not be included since it's change does not affect any plot at all
plt.figure(figsize=(25,25))
for i,var in enumerate(variable):
    plt.subplot(9,4,i+1)
    sns.distplot(df[var],bins=20)
    sns.distplot(df3[var],bins=20)
    

In [ ]:
categoricaldata= df3.select_dtypes(include=['object']).columns
categoricaldata

In [ ]:
pd.concat([df['MSZoning'].value_counts()/df.shape[0] *100,df3['MSZoning'].value_counts()/df.shape[0] *100],axis=1,keys=['MSZoning_org','MSZoning'])
#WE can see for categorical data MSZoning, we can see not much data has been lost

In [ ]:
#Now we will do the same for all other categorical variables
def cat_var_dist(var):
    return pd.concat([df[var].value_counts()/df.shape[0] *100,df3[var].value_counts()/df.shape[0] *100],axis=1,keys=[var+'_org',var+'_clean'])
    
    

In [ ]:
cat_var_dist('Street')

***CLEANING THE DATA-METHOD 2-IMPUTATION OF MISSING DATA VALUES BY MEAN,MEDIAN,MODE***

In [ ]:
#FIRST WE CHECK THE PERCENTAGE OF NULL VALUE IN THE ORIFINAL DATA FRAME DF
missing_val_per=df.isnull().sum()/df.shape[0]*100
missing_val_per

In [ ]:
#Again drop those columns whose percentage of null values is greater than 20
drop_columns  = null_var[null_var >17].keys()
drop_columns

In [ ]:
dfnew= df.drop(columns=drop_columns)
dfnew.shape

In [ ]:
#Now, we will select all the numeric data from the data frame
numeric_data=df.select_dtypes(['int64','float64'])
numeric_data.head(5)

In [ ]:
#now we will check for a missing data in our numeric data using visualization
plt.figure(figsize=(16,9))
sns.heatmap(numeric_data.isnull())
#in the heatmap, you can see that there are three columns who have missing values

In [ ]:
numeric_data[numeric_data.isnull().any(axis=1)]
#this will give all rows which have missing values

In [ ]:
numeric_data.isnull().sum()

In [ ]:
#in cases of dataframe with many columns we can do create a list of columns who have null values in the follwoing manner
missing_num_var=[var for var in numeric_data.columns if numeric_data[var].isnull().sum()>0]
missing_num_var

In [ ]:
#Now we have the data columns which have missing values.
#To decide if the missing values will be filled with mean or median, we have to see their distribution
plt.figure(figsize=(10,10))
sns.set()
for i,var in enumerate(missing_num_var):
    plt.subplot(2,2,i+1)
    sns.distplot(numeric_data[var],bins=20, kde_kws={'linewidth':5, 'color': '#DC143C'})

In [ ]:
#We will now replace the values with the mean of the data 
newnumeric_data=numeric_data.fillna(numeric_data.mean())
newnumeric_data.isnull().sum().sum()

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
for i,var in enumerate(missing_num_var):
    plt.subplot(2,2,i+1)
    sns.distplot(numeric_data[var],bins=20, kde_kws={'linewidth':8, 'color': 'red'},label='Original')
    sns.distplot(newnumeric_data[var],bins=20, kde_kws={'linewidth':5, 'color': 'green'},label='Mean')
    plt.legend()

In [ ]:
#We will now replace the values with the median of the data 
newnumeric_data_median=numeric_data.fillna(numeric_data.median())
newnumeric_data_median.isnull().sum().sum()

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
for i,var in enumerate(missing_num_var):
    plt.subplot(2,2,i+1)
    sns.distplot(numeric_data[var],bins=20, kde_kws={'linewidth':8, 'color': 'red'},label='Original')
    sns.distplot(newnumeric_data_median[var],bins=20, kde_kws={'linewidth':5, 'color': 'green'},label='Median')
    plt.legend()

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
for i,var in enumerate(missing_num_var):
    plt.subplot(2,2,i+1)
    sns.distplot(numeric_data[var],bins=20,hist=False, kde_kws={'linewidth':8, 'color': 'red'},label='Original')
    sns.distplot(newnumeric_data_median[var],bins=20,hist=False, kde_kws={'linewidth':5, 'color': 'black'},label='Median')
    sns.distplot(newnumeric_data[var],bins=20,hist=False, kde_kws={'linewidth':5, 'color': 'green'},label='Mean')
    plt.legend()

In [ ]:
#We will now create a boxplot to check for any outliers in out data set after mean,median imputation
for i,var in enumerate(missing_num_var):
    plt.figure(figsize=(10,10))
    plt.subplot(3,1,1)
    sns.boxplot(numeric_data[var])
    plt.subplot(3,1,2)
    sns.boxplot(newnumeric_data[var])
    plt.subplot(3,1,3)
    sns.boxplot(newnumeric_data_median[var])
    

**IMPUTATION WITH MEAN,MEDIAN,MODE FOR EACH CLASS**

In [ ]:
#Here, we will work with the original data frame(df) again.
##First, we will check the shape of the data
#Second,we will drop those columns who have more than 20 percentage of missing values.
#Third, we will only work with numeric data hence select those columns only
#All this has been done in the above cell, the numeric data has been saved under the name numeric data already
numeric_data.shape

In [ ]:
numeric_data.isnull().sum()

In [ ]:
#We have already saved the attributes which have missing values under variable name missing_num_var
missing_num_var

In [ ]:
df['LotConfig'].unique()

In [ ]:
#dfdf.loc[:,'LotConfig']=='Inside']['LotFrontage']

In [ ]:
#We will replace the NAN values with the mean of these numeric values using for loop in each category corresponding to LotConfig
#df_copy=df.copy()
#for var_class in df['LotConfig'].unique():
    #df_copy.update(df[df.loc[:,'LotConfig']==var_class]['LotFrontage'].replace(np.nan,df[df.loc[:,'LotConfig']==var_class]['LotFrontage'].mean()))

In [ ]:
#We will now write a code in such a way that in one loop only, all three columns are cleaned.
#We need to find categorical columns corresponding to each of the cikumns that need to be filled.
df_copy=df.copy()
num_var_miss=['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
cat_var=['LotConfig','Exterior2nd','KitchenQual']

for cat_var,num_var_miss in zip(cat_var,num_var_miss):
    for var_class in df[cat_var].unique():
        df_copy.update(df[df.loc[:,cat_var]==var_class][num_var_miss].replace(np.nan,df[df.loc[:,cat_var]==var_class][num_var_miss].mean()))

In [ ]:
df_copy[missing_num_var].isnull().sum()

In [ ]:
#We will now replace missing values with median
df_copy_median=df.copy()
num_var_miss=['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
cat_var=['LotConfig','Exterior2nd','KitchenQual']
for cat_var,num_var_miss in zip(cat_var,num_var_miss):
    for var_class in df[cat_var].unique():
        df_copy_median.update(df[df.loc[:,cat_var]==var_class][num_var_miss].replace(np.nan,df[df.loc[:,cat_var]==var_class][num_var_miss].median()))

In [ ]:
df_copy[missing_num_var].isnull().sum()

**DATA DISTRIBUTION**

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
for i,var in enumerate(missing_num_var):
    plt.subplot(2,2,i+1)
    sns.distplot(df[var],bins=20,hist=False, kde_kws={'linewidth':8, 'color': 'red'},label='Original')
    sns.distplot(df_copy[var],bins=20,hist=False, kde_kws={'linewidth':5, 'color': 'green'},label='Mean')
    sns.distplot(df_copy_median[var],bins=20,hist=False, kde_kws={'linewidth':5, 'color': 'black'},label='Median')
    plt.legend()

In [ ]:
#We will now create a boxplot to check for any outliers in out data set after mean,median imputation
for i,var in enumerate(missing_num_var):
    plt.figure(figsize=(10,10))
    plt.subplot(3,1,1)
    sns.boxplot(df[var])#original data
    plt.subplot(3,1,2)
    sns.boxplot(df_copy[var])#after mean imputation
    plt.subplot(3,1,3)
    sns.boxplot(df_copy_median[var])#after median imputation
    

****MISSING VALUE IMPUTATION OF CATEGORICAL VARIABLE****

In [ ]:
#We will use the original dataframe df here 
cat_variables=df.select_dtypes(include='object')#Storing the categoriacal variables
cat_variables.shape

In [ ]:
#Now we will check the percentage of data missing in categorical variables
cat_var_miss= cat_variables.isnull().mean()*100
cat_var_miss

In [ ]:
#we will drop columns with more than 20% of missing data
drop_columns  = cat_var_miss[cat_var_miss >17].keys()
drop_columns

In [ ]:
cat_variables=cat_variables.drop(columns=drop_columns)
cat_variables.shape

In [ ]:
#NOw we will find the cloumns who have more than zero percent missing values and impute them
isnull_per=cat_variables.isnull().mean()*100
miss_val=isnull_per[isnull_per>0].keys()
miss_val

**IMPUTATION WITH MODE **

In [ ]:
for var in miss_val:
    cat_variables[var].fillna(cat_variables[var].mode()[0],inplace = True)

In [ ]:
cat_variables.isnull().sum()

In [ ]:
plt.figure(figsize=(16,9))
for i,var in enumerate(miss_val):
    plt.subplot(4,3,i+1)
    plt.hist(cat_variables[var],label='Impute')
    plt.hist(df[var].dropna(),label='Original')
    plt.legend()